In [0]:
%sh sudo apt-get update && apt-get install -y gdal-bin libeccodes-dev libnetcdf-dev libhdf5-dev

In [0]:
%pip install numpy==1.24.4 eccodes==1.7.1 xarray==2024.9.0 cf_xarray==0.10.0 cfgrib==0.9.14.1 netcdf4==1.7.2

In [0]:
%pip install --no-cache-dir --force-reinstall cfgrib==0.9.14.1 numpy==1.24.4

In [0]:
import os

os.environ["ENV"] = "prd"
os.environ["STAGE"] = "sandbox"

In [0]:
import sys
import os
import re

if (config_path := re.split(r"notebooks", os.getcwd())[0] + "config") not in sys.path:
    sys.path.append(config_path)

if (package_path := re.split(r"notebooks", os.getcwd())[0] + "weather_package") not in sys.path:
    sys.path.append(package_path)

In [0]:
#ruff: noqa: F401
from pyspark.sql import Row
from pyspark.sql.functions import col

# import grib2netcdf converter from weather package
from grib2netcdf.converter import Grib2NetCDFConverter
from config import Config

import time


In [0]:
target_path = (
    "/Volumes/cdp_prd_sandbox_catalog_01/weather_silver/silver/ecmwf_conversion_speed_test"
)

source_paths = {
    "C1": "/Volumes/cdp_prd_live_catalog_01/weather_bronze/bronze/ecmwf/C1/2025/03/26/06/C1S03260600032606011",
    "C2": "/Volumes/cdp_prd_live_catalog_01/weather_bronze/bronze/ecmwf/C2/2025/03/26/06/C2S03260600032606011",
    "C8": "/Volumes/cdp_prd_live_catalog_01/weather_bronze/bronze/ecmwf/C8/2025/03/26/06/C8S03260600032606011",
    "C9": "/Volumes/cdp_prd_live_catalog_01/weather_bronze/bronze/ecmwf/C9/2025/03/26/06/C9S03260600032606011",
    "E1": "/Volumes/cdp_prd_live_catalog_01/weather_bronze/bronze/ecmwf/E1/2025/03/26/06/E1X03260600032606001",
    "E2": "/Volumes/cdp_prd_live_catalog_01/weather_bronze/bronze/ecmwf/E2/2025/03/26/06/E2X03260600032606001",
    "E8": "/Volumes/cdp_prd_live_catalog_01/weather_bronze/bronze/ecmwf/E8/2025/03/26/06/E8X03260600032606001",
    "E9": "/Volumes/cdp_prd_live_catalog_01/weather_bronze/bronze/ecmwf/E9/2025/03/26/06/E9X03260600032606001",
}

In [0]:
converter = Grib2NetCDFConverter(
    data_provider="ecmwf",
)

In [0]:
import pandas as pd

# Store rows for the table
results = []

print("Starting GRIB to NetCDF conversion...")

for code, path in source_paths.items():
    file_size = dbutils.fs.ls(path)[0].size / 1024 / 1024
    start_ts = time.time()
    converter.convert(path, f"{target_path}/{code}")
    stop_ts = time.time()

    file_name = path.split("/")[-1]
    conversion_time = stop_ts - start_ts

    results.append(
        {
            "Product Code": code,
            "Sample File Name": file_name,
            "File Size (MB)": round(file_size, 2),
            "Conversion Time (sec/file)": round(conversion_time, 3),
        }
    )
    print(f"Converted {code}")

# Create DataFrame and display it
df = pd.DataFrame(results)
display(df)


In [0]:
df

In [0]:
def list_all_files_recursively(path):
    files = []
    dirs_to_explore = [path]

    while dirs_to_explore:
        current_dir = dirs_to_explore.pop()
        items = dbutils.fs.ls(current_dir)

        for item in items:
            if item.isDir():
                dirs_to_explore.append(item.path)
            else:
                files.append(item.path)

    return files


# Usage
path = "/Volumes/cdp_prd_sandbox_catalog_01/weather_silver/silver/ecmwf_conversion_speed_test/E1/ECENSEU/"
all_files = list_all_files_recursively(path)
print(f"Total files found: {len(all_files)}")

results = []
print("Source GRIB file: E1X03260600032606001	137.00 MB")
for f in all_files:
    file_name = f.split("/")[-1]
    print(file_name)

    file_size = dbutils.fs.ls(f)[0].size / 1024 / 1024

    print(f"{file_name} - {file_size} MB")

    # results.append({
    #     "Product Code": code,
    #     "Sample File Name": file_name,
    #     "File Size (MB)": round(file_size, 2),
    #     "Conversion Time (sec/file)": round(conversion_time, 3)
    # })

# Create DataFrame and display it
# df = pd.DataFrame(results)

In [0]:
for f in all_files:
    print(f.split("/")[-1])

In [0]:
import xarray as xr
import matplotlib.pyplot as plt

# Open the NetCDF file
path = "/Volumes/cdp_prd_live_catalog_01/weather_silver/silver/ecmwf/ECCTRLEU/tas/2025/04/07/18/ECCTRLEU_0p10_tas_e000_2025040718z_2025040800z_h006.nc"
dataset = xr.open_dataset(path)

# Select variable and time slice
var = dataset["tas"].isel(valid_time=0, realization=0)

# Plot using matplotlib
plt.figure(figsize=(10, 6))
im = plt.pcolormesh(var["longitude"], var["latitude"], var, cmap="viridis", shading="auto")
plt.colorbar(im, label=var.units)
plt.title("Near-Surface Air Temperature")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(True)
plt.show()


In [0]:
import xarray as xr
import matplotlib.pyplot as plt

# Open the NetCDF file
path = "/Volumes/cdp_prd_live_catalog_01/weather_silver/silver/ecmwf/ECCTRLEU/tcc/2025/04/07/12/ECCTRLEU_0p10_tcc_e000_2025040712z_2025040722z_h010.nc"
ds = xr.open_dataset(path)

# Select variable and time slice
var = ds["tcc"].isel(valid_time=0, realization=0)

# Plot using matplotlib
plt.figure(figsize=(10, 6))
im = plt.pcolormesh(var["longitude"], var["latitude"], var, cmap="viridis", shading="auto")
plt.colorbar(im, label=var.units)
plt.title("mslp")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(True)
plt.show()


In [0]:
import xarray as xr
import matplotlib.pyplot as plt

# Open the NetCDF file
path = "/Volumes/cdp_prd_live_catalog_01/weather_silver/silver/ecmwf/ECENSEU/itssrd/2025/03/27/06/ECENSEU_0p10_itssrd_eALL_2025032706z_2025032707z_h001.nc"
dataset = xr.open_dataset(path)

# Select variable and time slice
var = dataset["itssrd"].isel(valid_time=0, realization=0)

# Plot using matplotlib
plt.figure(figsize=(10, 6))
im = plt.pcolormesh(var["longitude"], var["latitude"], var, cmap="viridis", shading="auto")
plt.colorbar(im, label=var.units)
plt.title("mslp")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.grid(True)
plt.show()


In [0]:
netcdf_file_path = "/Volumes/cdp_prd_live_catalog_01/weather_silver/silver/ecmwf/ECENSEU/itssrd/2025/03/27/06/ECENSEU_0p10_itssrd_eALL_2025032706z_2025032707z_h001.nc"

In [0]:
dataset = xr.open_dataset(netcdf_file_path)
dataset